- catalog of logM* and logMvir
- M*= 8.176+-0.2
- 2000 samples

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import corner
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic
import math
import pandas as pd

import sys 
sys.path.append('..')
from functions import *
# from colormaps import new_colors
cmap = plt.get_cmap('tab10')
new_colors = cmap(range(10))

3.0rc2


In [5]:
best_params = [0.31778976, 0.16864865, 2.76285812, 11.15042856, 0.12689323, 0.96288237,  -0.03033116]

In [6]:
run='UM_1'

config = parse_config('/Users/fardila/Documents/GitHub/dwarf_lensing/MCMC/config/mcmc_config_{0}.yaml'.format(str(run)))
config, cosmos_data, sim_data = initial_model(config)


5SHMR+scatter (Vmax@Mpeak)


In [7]:
def make_catalog_given_params(params, run, config = None, cosmos_data = None, sim_data = None, 
                              reduced=False, write=False):

    if not config:
        config = parse_config('/Users/fardila/Documents/GitHub/dwarf_lensing/MCMC/config/mcmc_config_{0}.yaml'.format(str(run)))
        config, cosmos_data, sim_data = initial_model(config)

    halo_data = sim_data['halocat'].halo_table
    
    stellar_masses = get_chris_stellar_masses(params, config, sim_data)
    halo_data['M_*'] = 10**stellar_masses
    
    if reduced:
        halo_data = halo_data['M_*','halo_mpeak','halo_mvir','halo_Vmax@Mpeak', 'halo_upid']
    
    if write:
        halo_data.write('/Users/fardila/Documents/GitHub/dwarf_lensing/mock_dwarfs/best_fit_mock_{0}.ascii'.format(run),
                  format ='ascii', overwrite=True, comment='#')
    
    return halo_data

In [8]:
make_catalog_given_params(best_params, 'UM_1', config, cosmos_data, sim_data)

halo_upid,halo_y,halo_x,halo_z,halo_Vmax@Mpeak,halo_rvir,halo_vy,halo_id,halo_vx,halo_mpeak,halo_vz,halo_mvir,halo_hostid,halo_mvir_host_halo,M_*
int64,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32,float32,int64,float32,float64
-1,23.92772,15.11237,39.99381,682.36,0.89861,271.35,2402494409,-101.65,98690000000000.0,-501.31,98690000000000.0,2402494409,98690000000000.0,147467497224.44037
2402494409,23.706131,15.139074,40.11049,111.81,0.020232404,126.53238,117000000134642248,3.06712,40340000000.0,-455.33575,1126000000.0,2402494409,98690000000000.0,8302065936.112039
2402494409,23.978075,15.21314,39.88245,118.21,0.019643746,-116.73203,112000000220447759,-390.76654,71320000000.0,-296.938,1031000000.0,2402494409,98690000000000.0,6780403269.998142
2402494409,23.779465,14.620061,40.39995,129.88,0.047781996,606.4685,95000000587913536,-115.19058,132900000000.0,-797.4637,14840000000.0,2402494409,98690000000000.0,3505323886.8661094
2402494409,23.69888,15.032578,40.70047,86.1,0.053081777,85.496735,9000002264306060,-153.64648,74750000000.0,1.4187,20340000000.0,2402494409,98690000000000.0,539210459.9326783
2402494409,24.54076,14.73342,39.95868,54.89,0.03473,948.18,2402494410,-285.0,16870000000.0,-628.17,5697000000.0,2402494409,98690000000000.0,116239748.90885982
2402494409,24.08037,14.98934,39.8334,32.0,0.033778,167.76,2402494411,762.84,9116000000.0,411.97,5241000000.0,2402494409,98690000000000.0,103211.56763808362
2402494409,24.49477,14.85569,39.55379,44.58,0.027928,741.17,2402494412,-118.65,7976000000.0,-360.0,2963000000.0,2402494409,98690000000000.0,1334245.6620525068
2402494409,24.4817,14.8217,39.76086,36.62,0.026416,869.4,2402494413,-26.35,3874000000.0,-1179.98,2507000000.0,2402494409,98690000000000.0,292638.29980271315


In [9]:
M_ddd =  8.176
two_sigma_ddd = 0.2
three_sigma_ddd = 0.3

def make_catalog_for_Asher(params, run, config = None, cosmos_data = None, sim_data = None, 
                              reduced=False, write=False):
    catalog = make_catalog_given_params(params, run, config, cosmos_data, sim_data)
    
    catalog = catalog[(np.log10(catalog['M_*'])>M_ddd-three_sigma_ddd) & (np.log10(catalog['M_*'])<M_ddd+three_sigma_ddd)]
    
    catalog['logM*'] = np.log10(catalog['M_*'])
    catalog['logMvir'] = np.log10(catalog['halo_mvir'])
    catalog['logMpeak'] = np.log10(catalog['halo_mvir'])
    
#     print catalog['logM*', 'logMvir'][:10]
    
    return catalog['logM*', 'logMvir','logMpeak']
    
    
    
    

In [10]:
def params_from_MCMC(run, steps):
    
    file_name = '/Users/fardila/Documents/GitHub/dwarf_lensing/MCMC/outfiles/cosmos_bplanck_run_{0}.hdf5'.format(str(run))
    reader = emcee.backends.HDFBackend(file_name)

    chains = reader.get_chain()
    log_prob = reader.get_log_prob()
   
    return chains[-steps:]

In [11]:
def create_catalog_for_n_steps(n_steps):
    f=open('/Users/fardila/Documents/GitHub/dwarf_lensing/mock_dwarfs/ddd_masses{0}.ascii'.format(n_steps),'ab')
    f.write('logM* \t logMvir \n')

    i=1
    
    appended_catalog = pd.DataFrame(columns=['logM*', 'logMvir'])

    for step in params_from_MCMC('UM_1', n_steps):
        print "step ", i
        i+=1
        for walker_params in step:
            print(walker_params)
            
            catalog = make_catalog_for_Asher(walker_params, 'UM_1', config, cosmos_data, sim_data)
            catalog = catalog.to_pandas()
            
            appended_catalog = appended_catalog.append(catalog)
            appended_catalog.to_pickle('/Users/fardila/Documents/GitHub/dwarf_lensing/mock_dwarfs/ddd_masses{0}.pkl'.format(n_steps))

#         if not step%100:
#             print('saving')
#             appended_catalog.to_pickle('/Users/fardila/Documents/GitHub/dwarf_lensing/mock_dwarfs/ddd_masses{0}.pkl'.format(n_steps))
    return appended_catalog

In [12]:
def create_catalogs_for_n_steps(n_steps):
    
    i=0

    for step in params_from_MCMC('UM_1', n_steps):
        print "step ", i
        
        walker_i = 0 
        for walker_params in step:
            print(walker_params)
            
            catalog = make_catalog_for_Asher(walker_params, 'UM_1', config, cosmos_data, sim_data)
            catalog = catalog.to_pandas()

            catalog.to_pickle('/Users/fardila/Documents/GitHub/dwarf_lensing/mock_dwarfs/ddd_masses{0}_{1}.pkl'.format(i, walker_i))
            walker_i+=1
            
        i+=1
    return

In [68]:
create_catalog_for_n_steps(10)

step  1
[ 0.4430145   0.33673792  2.76489094 10.99300189  0.13230935  1.05186809
  0.30996025]
[ 0.54210327  0.10906391  2.75577455 11.01913249  0.12460568  1.06016039
 -0.14315031]
[ 0.35751885  0.14790843  2.77412001 11.27764489  0.12475575  0.39983372
  0.54083326]
[ 0.46481277  0.13629336  2.75857417 11.05625902  0.12597793  1.05218429
 -0.15312206]
[ 0.54289055  0.21092098  2.75142637 11.06049321  0.12231269  0.95262413
 -0.391102  ]
[ 0.24667392  0.21493644  2.78070658 11.22169061  0.13023831  0.90857503
  0.06599865]
[ 0.1514319   0.13428359  2.78791143 11.42610182  0.12596015  0.93448896
 -0.60356603]
[ 0.16753045  0.1987855   2.77962276 11.27386036  0.12835103  0.80273076
  0.16207381]
[ 0.24464542  0.11927939  2.79562386 11.32638706  0.13038274  0.30576863
  0.78802292]
[ 1.43575147  0.45269359  2.70615331 10.93312785  0.10544272  0.21466474
 -0.03087336]
[ 0.50592192  0.12105894  2.76292319 11.00624612  0.1282333   0.51245932
  0.77014812]
[ 0.29469209  0.11635609  2.7840348

[ 0.54289055  0.21092098  2.75142637 11.06049321  0.12231269  0.95262413
 -0.391102  ]
[ 0.10704693  0.17381034  2.78543278 11.33244254  0.12852668  0.91271013
 -0.07391397]
[ 0.1514319   0.13428359  2.78791143 11.42610182  0.12596015  0.93448896
 -0.60356603]
[ 0.16753045  0.1987855   2.77962276 11.27386036  0.12835103  0.80273076
  0.16207381]
[ 0.24464542  0.11927939  2.79562386 11.32638706  0.13038274  0.30576863
  0.78802292]
[ 1.43575147  0.45269359  2.70615331 10.93312785  0.10544272  0.21466474
 -0.03087336]
[ 0.50592192  0.12105894  2.76292319 11.00624612  0.1282333   0.51245932
  0.77014812]
[ 0.29469209  0.11635609  2.78403486 11.32931474  0.12663068  0.09114733
  0.94492588]
[ 0.06099483  0.01856129  2.76547106 11.32473432  0.12271836  0.81463095
 -0.15669907]
[ 0.6060442   0.11965629  2.7527951  11.03360322  0.1215419   1.03329664
 -0.58072814]
[ 0.30916004  0.16144246  2.77269359 11.26200441  0.12547717  0.17811676
  0.88359752]
step  8
[ 0.4430145   0.33673792  2.7648909

,logM*,logMvir
0,8.122247,9.357744
1,7.979130,9.357744
2,8.178986,9.636388
3,8.213358,9.533772
4,8.054008,10.029790
5,8.306352,10.010724
6,8.223302,9.781827
7,8.225864,9.135769
8,8.015551,9.373096
9,8.069948,9.700098


In [71]:
catalog = make_catalog_for_Asher(best_params, 'UM_1', config, cosmos_data, sim_data)
catalog = catalog.to_pandas()
catalog.to_pickle('/Users/fardila/Documents/GitHub/dwarf_lensing/mock_dwarfs/ddd_masses_best_params.pkl')


In [72]:
catalog

,logM*,logMvir
0,8.283594,9.849113
1,8.239009,10.029790
2,8.123084,10.082067
3,8.151091,10.121231
4,8.295942,9.890980
5,8.251563,9.471731
6,8.004470,9.588160
7,8.173255,9.570776
8,8.251205,9.849113
9,8.090102,9.834866


In [16]:
create_catalogs_for_n_steps(100)

step  0
[ 0.2454407   0.45238568  2.76271323 11.06846052  0.13173387  1.22737009
 -0.12147974]
[ 0.41561246  0.19065661  2.76224165 11.16263931  0.12533205  0.66440419
  0.27016127]
[ 0.24599384  0.06448543  2.7849711  11.36204935  0.12621141  0.3601173
  0.61654145]
[ 0.46481277  0.13629336  2.75857417 11.05625902  0.12597793  1.05218429
 -0.15312206]
[ 0.47055041  0.15344561  2.74635908 11.03224521  0.12247453  0.70057547
  0.32093378]
[ 0.40261476  0.08010457  2.76497084 11.20173665  0.12417896  0.48729769
  0.55108136]
[ 0.18325895  0.11910454  2.78145514 11.36887153  0.12557728  0.94859407
 -0.57121411]
[ 0.16753045  0.1987855   2.77962276 11.27386036  0.12835103  0.80273076
  0.16207381]
[ 0.24464542  0.11927939  2.79562386 11.32638706  0.13038274  0.30576863
  0.78802292]
[ 1.46440181  0.43403996  2.70032159 10.92525498  0.10358544  0.24856167
 -0.14233838]
[ 0.33437721  0.09386897  2.7907403  11.26683545  0.13037036  0.02383898
  1.14329244]
[ 0.29469209  0.11635609  2.78403486

[ 0.53556594  0.19387757  2.74366214 10.97424383  0.12285434  1.02908418
 -0.21719444]
[ 0.40261476  0.08010457  2.76497084 11.20173665  0.12417896  0.48729769
  0.55108136]
[ 0.18325895  0.11910454  2.78145514 11.36887153  0.12557728  0.94859407
 -0.57121411]
[ 0.16753045  0.1987855   2.77962276 11.27386036  0.12835103  0.80273076
  0.16207381]
[ 0.24464542  0.11927939  2.79562386 11.32638706  0.13038274  0.30576863
  0.78802292]
[ 1.46440181  0.43403996  2.70032159 10.92525498  0.10358544  0.24856167
 -0.14233838]
[ 0.50145487  0.21237212  2.76375978 11.03201107  0.12819172  0.36070234
  0.89982505]
[ 0.29469209  0.11635609  2.78403486 11.32931474  0.12663068  0.09114733
  0.94492588]
[ 0.06099483  0.01856129  2.76547106 11.32473432  0.12271836  0.81463095
 -0.15669907]
[ 0.20307279  0.18980837  2.77119845 11.26958226  0.12620778  0.99129194
 -0.37566093]
[ 0.22659847  0.22462592  2.75956266 11.24356878  0.12328364  0.5897256
  0.33399405]
step  7
[ 0.27685007  0.43835132  2.76510327

[ 0.24464542  0.11927939  2.79562386 11.32638706  0.13038274  0.30576863
  0.78802292]
[ 1.46440181  0.43403996  2.70032159 10.92525498  0.10358544  0.24856167
 -0.14233838]
[ 0.45232011  0.14515848  2.74983373 10.98790134  0.12591894  0.46802548
  0.90783566]
[ 0.29469209  0.11635609  2.78403486 11.32931474  0.12663068  0.09114733
  0.94492588]
[ 0.06099483  0.01856129  2.76547106 11.32473432  0.12271836  0.81463095
 -0.15669907]
[ 0.20307279  0.18980837  2.77119845 11.26958226  0.12620778  0.99129194
 -0.37566093]
[ 0.15669635  0.18014907  2.77912978 11.36797403  0.12578967  0.39247946
  0.6060324 ]
step  13
[ 0.27905298  0.34917373  2.76058869 11.10034978  0.12953895  1.20566799
 -0.23293142]
[ 0.33060498  0.22552937  2.76706864 11.18029307  0.12698687  0.95682199
 -0.03035607]
[ 0.24599384  0.06448543  2.7849711  11.36204935  0.12621141  0.3601173
  0.61654145]
[ 0.46481277  0.13629336  2.75857417 11.05625902  0.12597793  1.05218429
 -0.15312206]
[ 0.53556594  0.19387757  2.7436621

[ 0.06099483  0.01856129  2.76547106 11.32473432  0.12271836  0.81463095
 -0.15669907]
[ 0.1839128   0.19844586  2.78290766 11.3189486   0.12834154  0.83105933
 -0.05008876]
[ 0.15669635  0.18014907  2.77912978 11.36797403  0.12578967  0.39247946
  0.6060324 ]
step  19
[ 0.27905298  0.34917373  2.76058869 11.10034978  0.12953895  1.20566799
 -0.23293142]
[ 0.1823716   0.14189872  2.78131814 11.35056517  0.12687519  0.66648451
  0.24544932]
[ 0.2139119   0.0599738   2.78598876 11.33093069  0.1274701   0.50948834
  0.53180607]
[ 0.46481277  0.13629336  2.75857417 11.05625902  0.12597793  1.05218429
 -0.15312206]
[ 0.48320043  0.2035161   2.75364192 11.02155073  0.12480826  1.16319182
 -0.56772504]
[ 0.45038686  0.12724731  2.75856548 11.07659014  0.12555665  0.75938077
  0.37800599]
[ 0.18325895  0.11910454  2.78145514 11.36887153  0.12557728  0.94859407
 -0.57121411]
[ 0.16753045  0.1987855   2.77962276 11.27386036  0.12835103  0.80273076
  0.16207381]
[ 0.24464542  0.11927939  2.795623

[ 0.1823716   0.14189872  2.78131814 11.35056517  0.12687519  0.66648451
  0.24544932]
[ 0.23046045  0.15046744  2.77657421 11.2326432   0.12842569  0.81888417
  0.23215523]
[ 0.46481277  0.13629336  2.75857417 11.05625902  0.12597793  1.05218429
 -0.15312206]
[ 0.48320043  0.2035161   2.75364192 11.02155073  0.12480826  1.16319182
 -0.56772504]
[ 0.41954727  0.14185051  2.76001312 11.07473298  0.12675277  0.85267242
  0.27480607]
[ 0.18325895  0.11910454  2.78145514 11.36887153  0.12557728  0.94859407
 -0.57121411]
[ 0.16753045  0.1987855   2.77962276 11.27386036  0.12835103  0.80273076
  0.16207381]
[ 0.24464542  0.11927939  2.79562386 11.32638706  0.13038274  0.30576863
  0.78802292]
[ 1.46440181  0.43403996  2.70032159 10.92525498  0.10358544  0.24856167
 -0.14233838]
[ 0.47411466  0.21089538  2.75708458 10.98072589  0.12830022  0.60223613
  0.74365818]
[ 0.29469209  0.11635609  2.78403486 11.32931474  0.12663068  0.09114733
  0.94492588]
[ 0.06099483  0.01856129  2.76547106 11.324

...

[ 0.50592192  0.12105894  2.76292319 11.00624612  0.1282333   0.51245932
  0.77014812]
[ 0.29469209  0.11635609  2.78403486 11.32931474  0.12663068  0.09114733
  0.94492588]
[ 0.06099483  0.01856129  2.76547106 11.32473432  0.12271836  0.81463095
 -0.15669907]
[ 0.63266753  0.1600182   2.74367948 10.97728693  0.11986631  1.16257914
 -0.88911882]
[ 0.3288008   0.25687201  2.7635526  11.13291835  0.12776171  0.67101015
  0.48923403]
step  69
[ 0.39711134  0.31942338  2.74664204 10.99905059  0.12676464  1.24959795
 -0.26374245]
[ 0.43799364  0.1060404   2.77395225 11.19914264  0.12630088  0.27948787
  0.85174635]
[ 0.40778775  0.17803799  2.76481377 11.07971808  0.12878611  0.77648843
  0.53609565]
[ 0.46481277  0.13629336  2.75857417 11.05625902  0.12597793  1.05218429
 -0.15312206]
[ 0.48320043  0.2035161   2.75364192 11.02155073  0.12480826  1.16319182
 -0.56772504]
[ 0.24667392  0.21493644  2.78070658 11.22169061  0.13023831  0.90857503
  0.06599865]
[ 0.19257849  0.1406916   2.780635

[ 0.30916004  0.16144246  2.77269359 11.26200441  0.12547717  0.17811676
  0.88359752]
step  75
[ 0.4430145   0.33673792  2.76489094 10.99300189  0.13230935  1.05186809
  0.30996025]
[ 0.47233427  0.09547403  2.76467456 11.13908766  0.12411476  0.95781839
 -0.23649251]
[ 0.35751885  0.14790843  2.77412001 11.27764489  0.12475575  0.39983372
  0.54083326]
[ 0.46481277  0.13629336  2.75857417 11.05625902  0.12597793  1.05218429
 -0.15312206]
[ 0.48320043  0.2035161   2.75364192 11.02155073  0.12480826  1.16319182
 -0.56772504]
[ 0.24667392  0.21493644  2.78070658 11.22169061  0.13023831  0.90857503
  0.06599865]
[ 0.19257849  0.1406916   2.78063563 11.36110036  0.12568986  0.97215965
 -0.59426484]
[ 0.16753045  0.1987855   2.77962276 11.27386036  0.12835103  0.80273076
  0.16207381]
[ 0.24464542  0.11927939  2.79562386 11.32638706  0.13038274  0.30576863
  0.78802292]
[ 1.43575147  0.45269359  2.70615331 10.93312785  0.10544272  0.21466474
 -0.03087336]
[ 0.50592192  0.12105894  2.762923

[ 0.46481277  0.13629336  2.75857417 11.05625902  0.12597793  1.05218429
 -0.15312206]
[ 0.54289055  0.21092098  2.75142637 11.06049321  0.12231269  0.95262413
 -0.391102  ]
[ 0.24667392  0.21493644  2.78070658 11.22169061  0.13023831  0.90857503
  0.06599865]
[ 0.1514319   0.13428359  2.78791143 11.42610182  0.12596015  0.93448896
 -0.60356603]
[ 0.16753045  0.1987855   2.77962276 11.27386036  0.12835103  0.80273076
  0.16207381]
[ 0.24464542  0.11927939  2.79562386 11.32638706  0.13038274  0.30576863
  0.78802292]
[ 1.43575147  0.45269359  2.70615331 10.93312785  0.10544272  0.21466474
 -0.03087336]
[ 0.50592192  0.12105894  2.76292319 11.00624612  0.1282333   0.51245932
  0.77014812]
[ 0.29469209  0.11635609  2.78403486 11.32931474  0.12663068  0.09114733
  0.94492588]
[ 0.06099483  0.01856129  2.76547106 11.32473432  0.12271836  0.81463095
 -0.15669907]
[ 0.6060442   0.11965629  2.7527951  11.03360322  0.1215419   1.03329664
 -0.58072814]
[ 0.30916004  0.16144246  2.77269359 11.262

[ 0.16753045  0.1987855   2.77962276 11.27386036  0.12835103  0.80273076
  0.16207381]
[ 0.24464542  0.11927939  2.79562386 11.32638706  0.13038274  0.30576863
  0.78802292]
[ 1.43575147  0.45269359  2.70615331 10.93312785  0.10544272  0.21466474
 -0.03087336]
[ 0.50592192  0.12105894  2.76292319 11.00624612  0.1282333   0.51245932
  0.77014812]
[ 0.29469209  0.11635609  2.78403486 11.32931474  0.12663068  0.09114733
  0.94492588]
[ 0.06099483  0.01856129  2.76547106 11.32473432  0.12271836  0.81463095
 -0.15669907]
[ 0.6060442   0.11965629  2.7527951  11.03360322  0.1215419   1.03329664
 -0.58072814]
[ 0.30916004  0.16144246  2.77269359 11.26200441  0.12547717  0.17811676
  0.88359752]
step  88
[ 0.4430145   0.33673792  2.76489094 10.99300189  0.13230935  1.05186809
  0.30996025]
[ 0.47233427  0.09547403  2.76467456 11.13908766  0.12411476  0.95781839
 -0.23649251]
[ 0.35751885  0.14790843  2.77412001 11.27764489  0.12475575  0.39983372
  0.54083326]
[ 0.46481277  0.13629336  2.758574

[ 0.29469209  0.11635609  2.78403486 11.32931474  0.12663068  0.09114733
  0.94492588]
[ 0.06099483  0.01856129  2.76547106 11.32473432  0.12271836  0.81463095
 -0.15669907]
[ 0.6060442   0.11965629  2.7527951  11.03360322  0.1215419   1.03329664
 -0.58072814]
[ 0.30916004  0.16144246  2.77269359 11.26200441  0.12547717  0.17811676
  0.88359752]
step  94
[ 0.4430145   0.33673792  2.76489094 10.99300189  0.13230935  1.05186809
  0.30996025]
[ 0.54210327  0.10906391  2.75577455 11.01913249  0.12460568  1.06016039
 -0.14315031]
[ 0.35751885  0.14790843  2.77412001 11.27764489  0.12475575  0.39983372
  0.54083326]
[ 0.46481277  0.13629336  2.75857417 11.05625902  0.12597793  1.05218429
 -0.15312206]
[ 0.54289055  0.21092098  2.75142637 11.06049321  0.12231269  0.95262413
 -0.391102  ]
[ 0.10704693  0.17381034  2.78543278 11.33244254  0.12852668  0.91271013
 -0.07391397]
[ 0.1514319   0.13428359  2.78791143 11.42610182  0.12596015  0.93448896
 -0.60356603]
[ 0.16753045  0.1987855   2.779622

In [15]:
unpickled_df = pd.read_pickle('/Users/fardila/Documents/GitHub/dwarf_lensing/mock_dwarfs/ddd_masses0_14.pkl')
unpickled_df

,logM*,logMvir,logMpeak
0,7.939236,9.357744,9.357744
1,8.464276,10.113609,10.113609
2,8.091510,10.010724,10.010724
3,7.936868,10.121231,10.121231
4,8.214598,9.991226,9.991226
5,7.920088,9.820136,9.820136
6,8.448215,9.781827,9.781827
7,8.431262,9.773201,9.773201
8,8.070465,9.894704,9.894704
9,8.296292,9.405518,9.405518
